In [67]:
from kingdon import Algebra
from sympy import ccode, symbols

In [68]:
alg = Algebra(2, 0, 1)

In [69]:
a = alg.multivector(name="a")
b = alg.multivector(name="b")

In [70]:
alg.bin2canon

{0: 'e', 1: 'e0', 2: 'e1', 3: 'e01', 4: 'e2', 5: 'e02', 6: 'e12', 7: 'e012'}

In [71]:
r = a * b

In [72]:
code = ccode(r.values()[0])
code

'a*b + a1*b1 - a12*b12 + a2*b2'

In [73]:
code.replace("a", "a.e").replace("b", "b.e")

'a.e*b.e + a.e1*b.e1 - a.e12*b.e12 + a.e2*b.e2'

In [74]:
# Convert multivector to C++ macro code
def mv_to_code(mv, name, op="+="):
    lines = []
    for key, value in zip(mv.keys(), mv.values()):
        key = "    r." + alg.bin2canon[key]
        value = ccode(value).replace("a", "a.e").replace("b", "b.e")
        line = key + f" {op} " + value + ";"
        lines.append(line)
    code = "\\\n".join(lines)
    code = "#define {} \\\n".format(name) + code
    return code

fragment = mv_to_code(r, "GP_full")
print(fragment)


#define GP_full \
    r.e += a.e*b.e + a.e1*b.e1 - a.e12*b.e12 + a.e2*b.e2;\
    r.e0 += a.e*b.e0 + a.e0*b.e + a.e01*b.e1 - a.e012*b.e12 + a.e02*b.e2 - a.e1*b.e01 - a.e12*b.e012 - a.e2*b.e02;\
    r.e1 += a.e*b.e1 + a.e1*b.e + a.e12*b.e2 - a.e2*b.e12;\
    r.e2 += a.e*b.e2 + a.e1*b.e12 - a.e12*b.e1 + a.e2*b.e;\
    r.e01 += a.e*b.e01 + a.e0*b.e1 + a.e01*b.e + a.e012*b.e2 - a.e02*b.e12 - a.e1*b.e0 + a.e12*b.e02 + a.e2*b.e012;\
    r.e02 += a.e*b.e02 + a.e0*b.e2 + a.e01*b.e12 - a.e012*b.e1 + a.e02*b.e - a.e1*b.e012 - a.e12*b.e01 - a.e2*b.e0;\
    r.e12 += a.e*b.e12 + a.e1*b.e2 + a.e12*b.e - a.e2*b.e1;\
    r.e012 += a.e*b.e012 + a.e0*b.e12 + a.e01*b.e2 + a.e012*b.e - a.e02*b.e1 - a.e1*b.e02 + a.e12*b.e0 + a.e2*b.e01;


In [75]:
def generate_product_op(name="GP", op=lambda a, b: a * b):
    frags = []
    for i in range(4):
        for j in range(4):
            a = alg.multivector(name="a", grades=(i,))
            b = alg.multivector(name="b", grades=(j,))
            r = op(a, b)
            frag = mv_to_code(r, "{}_{}_{}(r, a, b, op)".format(name, i, j), "op")
            frags.append(frag)

    return frags

def generate_product_op_full(name="GP", op=lambda a, b: a * b):
    frags = []
    a = alg.multivector(name="a")
    b = alg.multivector(name="b")
    r = op(a, b)
    frag = mv_to_code(r, "{}_FULL(r, a, b)".format(name), "=")
    frags.append(frag)

    return frags

def generate_additive_op(name="ADD", op=lambda a, b: a + b):
    frags = []
    for i in range(4):
        a = alg.multivector(name="a", grades=(i,))
        b = alg.multivector(name="b", grades=(i,))
        r = op(a, b)
        frag = mv_to_code(r, "{}_{}(r, a, b, op)".format(name, i), "op")
        frags.append(frag)

    return frags

def generate_scalar_op(name="Mul", op=lambda a, b: a + b):
    frags = []
    b = alg.scalar("b")
    for i in range(4):
        a = alg.multivector(name="a", grades=(i,))
        r = op(a, b)
        frag = mv_to_code(r, "{}_{}(r, a, b, op)".format(name, i), "op")
        frag = frag.replace("b.e", "b")
        frags.append(frag)

    return frags

def generate_unary_op(name="REVERSE", op=lambda a: a.reverse()):
    frags = []
    for i in range(4):
        a = alg.multivector(name="a", grades=(i,))
        r = op(a)
        frag = mv_to_code(r, "{}_{}(r, a)".format(name, i), "=")
        frags.append(frag)

    return frags

def generate_grade_op():
    frags = []
    for i in range(4):
        a = alg.multivector(name="a", grades=(i,))
        r = a
        frag = mv_to_code(r, "{}_{}(r, a)".format("GRADE", i), "=")
        frags.append(frag)

    return frags


def generate_fragments():
    frags = []

    # NO NECESSARY
    # Geometric Product in full
    # Outer Product in full
    # Inner Product in full
    # Regressive Product in full

    # TOO EXPENSIVE
    # Sandwitch(Conjugate) Product in full
    # Project in full

    # Geometric Product in grades
    frags += generate_product_op(name="GP", op=lambda a, b: a * b)
    frags += generate_product_op_full(name="GP", op=lambda a, b: a * b)

    # Outer Product in grades
    frags += generate_product_op(name="OP", op=lambda a, b: a ^ b)
    frags += generate_product_op_full(name="OP", op=lambda a, b: a ^ b)
    
    # Inner Product in grades
    frags += generate_product_op(name="IP", op=lambda a, b: a | b)
    frags += generate_product_op_full(name="IP", op=lambda a, b: a | b)

    # Regressive Product in grades
    # TODO: check is RP splitable
    frags += generate_product_op(name="RP", op=lambda a, b: a & b)
    frags += generate_product_op_full(name="RP", op=lambda a, b: a & b)
    
    # Commutator Product
    frags += generate_product_op(name="CP", op=lambda a, b: a.cp(b))
    frags += generate_product_op_full(name="CP", op=lambda a, b: a.cp(b))

    # Scalar Product
    frags += [frag.replace("r.e", "r") for frag
               in generate_product_op_full(name="SP", op=lambda a, b: a.sp(b))]
    
    
    # Reverse
    frags += generate_unary_op(name="REVERSE", op=lambda a: a.reverse())
    # Dual
    frags += generate_unary_op(name="DUAL", op=lambda a: a.dual())

    # Grade
    frags += generate_grade_op()
    # Inverse
    #frags += generate_unary_op(name="INVERSE", op=lambda a: a.inv())

    # Add
    frags += generate_additive_op(name="ADD", op=lambda a, b: a + b)
    # Sub
    frags += generate_additive_op(name="SUB", op=lambda a, b: a - b)
    # Scalar Mul
    frags += generate_scalar_op(name="MUL", op=lambda a, b: a * b)
    # Scalar Div
    frags += generate_scalar_op(name="DIV", op=lambda a, b: a / b)

    return "\n\n".join(frags)

header = """// Generate from https://github.com/xiongyaohua/PGA_codegen

"""
code = header + generate_fragments()
with open("pga2.inc", "w") as f:
    f.write(code)